In [43]:
import gym
import gym_jumping_task
from gym_jumping_task.envs.jumping_task import JumpTaskEnv
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid
from env import VanillaEnv
from rl.common.buffer2 import Transition, Episode, RolloutBuffer
from rl.ppo.policies import ActorCriticNet
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from typing import List

In [44]:
env = VanillaEnv(configurations=[
        (22, 8), (27, 8), (32, 8), (38, 8),
        (22, 20), (27, 20), (32, 20), (38, 20),
        (22, 32), (27, 32), (32, 32), (38, 32),
],rendering=True)

n_iterations = 10_00
gamma = 0.99

episodes: List[Episode] = []

for i in range(n_iterations):
    done = False
    episode = Episode(discount=gamma)
    obs = env.reset()
    obstacle_position = env.actualEnv.obstacle_position;
    jumping_pixel = obstacle_position-14;
    step = 0
    episode_reward = 0
    while not done:
        action = 1 if step==jumping_pixel else 0
        obs, reward, done, _ = env.step(action)
        episode.append(Transition(obs, action, reward, 0))
        env.render()
        step +=1
    episodes.append(episode)
    if i % 100==0: print(f"Episode {i} return {np.sum(episode.rewards())}")
    

Episode 0 return 156.0
Episode 100 return 156.0
Episode 200 return 156.0
Episode 300 return 156.0
Episode 400 return 156.0
Episode 500 return 156.0
Episode 600 return 156.0
Episode 700 return 156.0
Episode 800 return 156.0
Episode 900 return 156.0


In [45]:
# get the states, returns and actions out of the episodes
states, actions, values = [], [], []
for episode in episodes:
    states += episode.states()
    actions += episode.actions()
    values += episode.calculate_return()

states, actions, values = np.array(states), np.array(actions), np.array(values)
# vertically add actions and values
targets = np.column_stack((actions, values))

X = torch.tensor(states)
Y = torch.tensor(targets)

print(f"Shape of X: {X.shape} dtype: {X.dtype}")
print(f"Shape of Y: {Y.shape} dtype: {Y.dtype}")

Shape of X: torch.Size([56000, 1, 60, 60]) dtype: torch.uint8
Shape of Y: torch.Size([56000, 2]) dtype: torch.float64


In [52]:
class BCDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        assert x.shape[0] == y.shape[0]
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

data: BCDataset = BCDataset(X, Y)
train_set, val_set = torch.utils.data.random_split(data, [int(len(data)*0.8), int(len(data)*0.2)])

trainloader:DataLoader = DataLoader(train_set, batch_size=64, shuffle=True)
testloader:DataLoader = DataLoader(val_set, batch_size=64, shuffle=True)

created train loader and test loader with sizes 700 and 175 respectively


In [73]:
def train(net:ActorCriticNet, dataLoader: DataLoader, 
          optim_actor, optim_critic,
          loss_actor, loss_critic
          ):
    device = next(net.parameters()).device

    for batch in dataLoader:
        # X is the observation
        # 
        X, y = batch[0].to(device), batch[1].to(device)
        pred_action_logits = net.actor.forward(X)
        pred_values = net.critic.forward(X)

        actor_error = loss_actor(pred_action_logits, torch.tensor(y[:,0], dtype=torch.int64))
        critic_error = loss_critic(pred_values, y[:,1])

        optim_actor.zero_grad()
        actor_error.backward()
        optim_actor.step()

        optim_critic.zero_grad()
        critic_error.backward()
        optim_critic.step()




In [74]:
lr = 0.004

model = ActorCriticNet();
optim_actor = optim.Adam(model.actor.parameters(), lr=lr);
optim_critic = optim.Adam(model.critic.parameters(), lr=lr)
loss_actor = nn.CrossEntropyLoss()
loss_critic = nn.MSELoss()
                       
train(model, trainloader, optim_actor, optim_critic, loss_actor, loss_critic)

C:\Users\manus\AppData\Local\Temp\ipykernel_23480\3862726229.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  actor_error = loss_actor(pred_action_logits, torch.tensor(y[:,0], dtype=torch.int64))


RuntimeError: Found dtype Double but expected Float